### Discrete Rideshare Simulation using DIPR for Routing of Rides

The provided script simulates a ridesharing system. It involves creating a discrete, dynamic and adaptive environment where multiple agents (riders and drivers) interact over a grid-based city model. The primary focus is on optimizing the allocation of ride requests to drivers, taking into account both travel distance and dynamically updated congestion priorities.

#### Key Components

1. **Grid and Zones**:
   - The city is modeled as a grid of zones, represented as nodes in a directed graph. Roads between zones are represented as edges with associated weights.
   - Each zone tracks the length of its queue, which indicates the congestion level.

2. **Dynamic Priority Calculation**:
   - The system uses a discounted integral priority routing (DIPR) strategy to manage congestion. Each zone's priority is updated based on its historical queue lengths, giving more weight to recent data.
   - Priorities influence the cost of routing decisions, where higher congestion increases the travel cost to a zone.

3. **Event-Driven Simulation**:
   - The simulation is event-driven, handling various events such as ride requests, driver assignments, and ride completions.
   - An event queue manages the sequence and timing of events, ensuring that the system evolves over time.

4. **Agent-Based Modeling**:
   - The simulation includes two types of agents: riders and drivers. Each agent follows specific states and transitions, managed by state machines.
   - Drivers and riders interact through ride requests, with drivers being assigned to riders based on their state and preferences.

5. **Adaptive Routing**:
   - Routing decisions are made dynamically, incorporating both physical distance and congestion priorities. This balances the need to minimize travel distance with the goal of avoiding congested areas.
   - The system continuously updates priorities and routes based on real-time conditions.

6. **State Recording and Analysis**:
   - The simulation records the state of the system at each time step, capturing metrics such as completed rides, total distance traveled, and total cost.
   - These metrics are used to evaluate the performance of the system and compare different routing strategies.

7. **Output and Comparison**:
   - The state of the system is written to a CSV file, allowing for post-simulation analysis and comparison with other simulations.
   - The output data can be used to determine the efficiency and effectiveness of different strategies, such as traditional routing versus DIPR-based routing.


In [12]:
import csv
import time
import heapq
import random
from sys import argv
from math import ceil, floor
import networkx as nx
import heapq
from collections import deque, defaultdict, Counter
from transitions import Machine
from typing import List, Dict, Any
from dataclasses import dataclass, field
import pandas as pd

In [13]:
class Grid:
    def __init__(self, width=4, height=4, min_weight=1, max_weight=10, alpha=0.9):
        self.width = width
        self.height = height
        self.min_weight = min_weight
        self.max_weight = max_weight
        self.alpha = alpha
        self.graph = nx.DiGraph()
        self.zones = {}
        self.queues = defaultdict(int)
        self.priorities = defaultdict(int)
        self.create_graph()
        self.create_zones()
        self.precompute_shortest_paths()

    def create_graph(self):
        for i in range(self.width):
            for j in range(self.height):
                self.graph.add_node((i, j))
        for i in range(self.width):
            for j in range(self.height - 1):
                weight_1 = random.randint(self.min_weight, self.max_weight)
                weight_2 = random.randint(self.min_weight, self.max_weight)
                self.graph.add_edge((i, j), (i, j + 1), weight=weight_1)
                self.graph.add_edge((i, j + 1), (i, j), weight=weight_2)
                weight_3 = random.randint(self.min_weight, self.max_weight)
                weight_4 = random.randint(self.min_weight, self.max_weight)
                self.graph.add_edge((j, i), (j + 1, i), weight=weight_3)
                self.graph.add_edge((j + 1, i), (j, i), weight=weight_4)

    def create_zones(self):
        self.zones = {(i, j): Zone(i, j) for i in range(self.width) for j in range(self.height)}

    def precompute_shortest_paths(self):
        self.shortest_paths = dict(nx.all_pairs_dijkstra_path_length(self.graph, weight='weight'))

    def update_priorities(self):
        for node in self.graph.nodes:
            self.priorities[node] = self.alpha * self.priorities[node] + self.queues[node]      

class Zone:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.name = f"Zone ({x},{y})"
        self.queue_length = 0

    def __str__(self):
        return f"Zone ({self.x},{self.y})"

    def update_queue(self, length):
        self.queue_length = length

In [14]:
@dataclass(order=True)
class Event:
    t: int
    action: str
    spaces: Dict[str, Any] = field(compare=False)

class Simulation:
    def __init__(self, riders, drivers, ride_requests, grid):
        self.riders = riders
        self.drivers = drivers
        self.ride_requests = ride_requests
        self.grid = grid
        self.completed_rides = []
        self.state_map = StateMap()

    def calculate_cost(self, zone1, zone2):
        distance = self.grid.shortest_paths[(zone1.x, zone1.y)][(zone2.x, zone2.y)]
        priority = self.grid.priorities[(zone2.x, zone2.y)]
        lambda_factor = 1  # This factor can be adjusted as needed
        return distance + lambda_factor * priority
    
    def calculate_travel_time(self, zone1, zone2):
        try:
            return self.grid.shortest_paths[(zone1.x, zone1.y)][(zone2.x, zone2.y)]
        except KeyError:
            return float('inf')

    def handle_ride_request_event(self, event, event_queue, current_time, retry_queue):
        ride_id = event.spaces["ride_id"]
        ride_request = self.ride_requests[ride_id]
        print(f"Handling ride request event for ride {ride_id} at time {current_time}")

        for driver in self.drivers:
            if driver.state == 'idle' and driver.will_accept_ride(ride_request.metadata):
                travel_time = self.calculate_travel_time(driver.zone, ride_request.origin_zone)
                cost = self.calculate_cost(driver.zone, ride_request.origin_zone)
                print(f"Travel time: {travel_time}, Cost: {cost}")
                if travel_time < float('inf'):
                    driver.zone = ride_request.origin_zone
                    driver.time_to_next_zone = travel_time
                    driver.current_rider = ride_request.rider
                    driver.current_request = ride_request
                    driver.accept_ride()
                    ride_request.driver = driver
                    ride_request.accept()
                    ride_request.rider.match_with_driver()
                    ride_request.rider.estimated_wait_time = travel_time
                    ride_request.metadata["pickup_time"] = current_time + travel_time
                    heapq.heappush(event_queue, Event(t=current_time + travel_time, action="Driver Assigned", spaces={"ride_id": ride_id, "driver_id": driver.name}))
                    print(f"Driver {driver.name} assigned to ride {ride_id} at time {current_time + travel_time}")
                    return

        print(f"No driver available for ride {ride_id} at time {current_time}, retrying...")
        retry_queue.append(Event(t=current_time + 1, action="Request Ride", spaces={"ride_id": ride_id}))

    def handle_driver_assigned_event(self, event, event_queue, current_time):
        ride_id = event.spaces["ride_id"]
        driver_id = event.spaces["driver_id"]
        ride_request = self.ride_requests[ride_id]
        driver = next(d for d in self.drivers if d.name == driver_id)
        print(f"Driver {driver_id} arrived at pickup for ride {ride_id} at time {current_time}")

        driver.arrive_at_pickup()
        driver.pick_up_rider()
        ride_request.start()
        ride_request.rider.start_ride()
        travel_time = self.calculate_travel_time(driver.zone, ride_request.destination_zone)
        driver.time_to_next_zone = travel_time
        ride_request.metadata["dropoff_time"] = current_time + travel_time
        heapq.heappush(event_queue, Event(t=current_time + travel_time, action="Complete Ride", spaces={"ride_id": ride_id, "driver_id": driver.name}))
        print(f"Ride {ride_id} in progress, will complete at time {current_time + travel_time}")

    def handle_complete_ride_event(self, event, event_queue, current_time):
        ride_id = event.spaces["ride_id"]
        driver_id = event.spaces["driver_id"]
        ride_request = self.ride_requests[ride_id]
        driver = next(d for d in self.drivers if d.name == driver_id)
        print(f"Completing ride {ride_id} by driver {driver_id} at time {current_time}")

        driver.complete_ride()
        ride_request.complete()
        ride_request.rider.complete_ride()
        ride_request.metadata["dropoff_time"] = current_time
        driver.current_rider = None
        driver.current_request = None
        driver.reset()

        self.completed_rides.append(ride_request)
        print(f"Ride {ride_id} completed")
        
    def record_state(self, current_time):
        completed_rides = len(self.completed_rides)
        total_distance = sum([ride.metadata['distance_km'] for ride in self.completed_rides])
        total_cost = sum([ride.metadata['cost_usd'] for ride in self.completed_rides])
        self.state_map.record_state(current_time, completed_rides, total_distance, total_cost)

In [15]:

class Driver:
    states = ['idle', 'en_route', 'waiting_for_rider', 'in_ride', 'ride_completed']

    def __init__(self, name, zone, preferences):
        self.name = name
        self.zone = zone
        self.preferences = preferences
        self.time_to_next_zone = 0
        self.current_rider = None
        self.current_request = None
        self.machine = Machine(model=self, states=Driver.states, initial='idle')
        self.machine.add_transition('accept_ride', 'idle', 'en_route')
        self.machine.add_transition('arrive_at_pickup', 'en_route', 'waiting_for_rider')
        self.machine.add_transition('pick_up_rider', 'waiting_for_rider', 'in_ride')
        self.machine.add_transition('complete_ride', 'in_ride', 'ride_completed')
        self.machine.add_transition('reset', '*', 'idle')

    def calculate_acceptance_threshold(self, ride_metadata):
        threshold = (
            ride_metadata['cost_usd'] * self.preferences['cost_weight'] +
            ride_metadata['distance_km'] * self.preferences['distance_weight'] +
            ride_metadata['duration_min'] * self.preferences['duration_weight'] +
            ride_metadata['driver_rating'] * self.preferences['driver_rating_weight'] +
            ride_metadata['rider_rating'] * self.preferences['rider_rating_weight']
        )
        return threshold

    def will_accept_ride(self, ride_metadata):
        # acceptance_threshold = self.calculate_acceptance_threshold(ride_metadata)
        # return acceptance_threshold >= self.preferences['acceptance_threshold']
        return True

In [16]:
class Rider:
    states = ['waiting', 'matched', 'in_ride', 'ride_completed', 'ride_canceled']

    def __init__(self, name, zone):
        self.name = name
        self.zone = zone
        self.estimated_wait_time = 0
        self.machine = Machine(model=self, states=Rider.states, initial='waiting')
        self.machine.add_transition('match_with_driver', 'waiting', 'matched')
        self.machine.add_transition('start_ride', 'matched', 'in_ride')
        self.machine.add_transition('complete_ride', 'in_ride', 'ride_completed')
        self.machine.add_transition('cancel_ride', '*', 'ride_canceled')

In [17]:
class RideRequest:
    states = ['requested', 'accepted', 'in_progress', 'completed', 'canceled']

    def __init__(self, ride_id, rider, origin_zone, destination_zone, grid):
        self.ride_id = ride_id
        self.rider = rider
        self.origin_zone = origin_zone
        self.destination_zone = destination_zone
        self.grid = grid
        self.metadata = self.generate_metadata()
        self.driver = None
        self.machine = Machine(model=self, states=RideRequest.states, initial='requested')
        self.machine.add_transition('accept', 'requested', 'accepted')
        self.machine.add_transition('start', 'accepted', 'in_progress')
        self.machine.add_transition('complete', 'in_progress', 'completed')
        self.machine.add_transition('cancel', '*', 'canceled')

    def generate_metadata(self):
        driver_to_pickup = self.calculate_travel_time(self.origin_zone, self.origin_zone)
        pickup_to_destination = self.calculate_travel_time(self.origin_zone, self.destination_zone)
        total_distance = driver_to_pickup + pickup_to_destination
        duration = total_distance
        cost = total_distance * 2.5
        return {
            "distance_km": total_distance,
            "duration_min": duration,
            "pickup_time": None,
            "dropoff_time": None,
            "cost_usd": round(cost, 2),
            "driver_rating": round(random.uniform(4.0, 5.0), 2),
            "rider_rating": round(random.uniform(4.0, 5.0), 2),
            "ride_type": random.choice(["economy", "premium", "pool"]),
            "traffic_conditions": random.choice(["normal", "heavy", "light"]),
            "weather_conditions": random.choice(["clear", "rainy", "stormy", "snowy"]),
        }

    def calculate_travel_time(self, zone1, zone2):
        try:
            return self.grid.shortest_paths[(zone1.x, zone1.y)][(zone2.x, zone2.y)]
        except KeyError:
            return float('inf')

In [18]:
class StateMap:
    def __init__(self):
        self.states = []

    def record_state(self, current_time, completed_rides, total_distance, total_cost):
        state = {
            "time": current_time,
            "completed_rides": completed_rides,
            "total_distance": total_distance,
            "total_cost": total_cost
        }
        self.states.append(state)

    def print_state_map(self):
        for state in self.states:
            print(f"Time: {state['time']}, Completed Rides: {state['completed_rides']}, Total Distance: {state['total_distance']}, Total Cost: {state['total_cost']}")

    def write_to_csv(self, filename):
        with open(filename, 'w', newline='') as csvfile:
            fieldnames = ['time', 'completed_rides', 'total_distance', 'total_cost']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()
            for state in self.states:
                writer.writerow(state)

In [19]:
def initialize_event_queue(ride_requests) -> List[Event]:

    initial_events = []
    start_time_counter = Counter()

    for ride_id in ride_requests:
        start_time = random.randint(1, len(ride_requests))
        start_time_counter[start_time] += 1
        initial_events.append(Event(t=start_time, action="Request Ride", spaces={"ride_id": ride_id}))
    for _, count in list(start_time_counter.items()):
        if count > 10:
            start_time_counter[start_time] = 10
            
    heapq.heapify(initial_events)

    return initial_events

def process_events(event_queue, simulation):
    current_time = 0
    retry_queue = deque()

    while event_queue or retry_queue:
        while event_queue and event_queue[0].t == current_time:
            current_event = heapq.heappop(event_queue)
            handle_event(current_event, event_queue, simulation, retry_queue, current_time)
        while retry_queue and retry_queue[0].t <= current_time:
            current_event = retry_queue.popleft()
            handle_event(current_event, event_queue, simulation, retry_queue, current_time)
        
        # Record the state at each time step
        simulation.record_state(current_time)
        print("State recorded")
        current_time += 1
        simulation.grid.update_priorities()
    
    print(f"The event queue is now empty. Simulation complete! Time: {current_time}")
    simulation.state_map.print_state_map()
    
def handle_event(event, event_queue, simulation, retry_queue, current_time):
    print(f"Processing event: {event} at time {current_time}")
    
    if event.action == "Request Ride":
        simulation.handle_ride_request_event(event, event_queue, current_time, retry_queue)
    elif event.action == "Driver Assigned":
        simulation.handle_driver_assigned_event(event, event_queue, current_time)
    elif event.action == "Complete Ride":
        simulation.handle_complete_ride_event(event, event_queue, current_time)
    else:
        print(f"Unknown event action: {event.action}")

def adjust_inputs(seed=0, upper_limit=1000):
    base_width = 4
    base_height = 4
    base_weight = 2
    base_num_riders = 4

    seed = max(0, min(seed, 100))
    random.seed(time.time())

    increase_factor = 1 + (seed / 100) * 500
    random_factor = lambda: random.uniform(0.9, 1.1)

    grid_width = ceil(base_width * increase_factor * random_factor())
    grid_height = ceil(base_height * increase_factor * random_factor())
    grid_weight = ceil(base_weight * increase_factor * random_factor())
    num_riders = ceil(base_num_riders * increase_factor * random_factor())

    rider_driver_delta = random.randint(-100, -1)
    num_drivers = max(1, min(num_riders + rider_driver_delta, upper_limit))

    grid_width = max(2, min(grid_width, 15))
    grid_height = max(2, min(grid_height, 15))
    grid_weight = max(1, min(grid_weight, 10))
    num_riders = max(1, min(num_riders, upper_limit))
    num_drivers = max(1, min(num_drivers, num_riders - 1))

    grid = Grid(width=grid_width, height=grid_height)
    shortest_paths = grid.shortest_paths

    riders = [Rider(f'Rider {i}', random.choice(list(grid.zones.values()))) for i in range(num_riders)]
    drivers = [Driver(f'Driver {i}', random.choice(list(grid.zones.values())), {
        "cost_weight": 0.4, "distance_weight": 0.1, "duration_weight": 0.1,
        "driver_rating_weight": 0.2, "rider_rating_weight": 0.2, "acceptance_threshold": 20
    }) for i in range(floor(num_riders / 4))]

    ride_requests = {
        i + 1: RideRequest(i + 1, riders[i], random.choice(list(grid.zones.values())), random.choice(list(grid.zones.values())), grid)
        for i in range(num_riders)
    }

    return grid, riders, drivers, ride_requests


In [20]:
seed = 100
upper_limit = 75000

grid, riders, drivers, ride_requests = adjust_inputs(seed, upper_limit)

print("Grid dimensions:", grid.width, grid.height)
print("Number of Zones:", len(grid.zones))
print("Number of Nodes:", str(grid.graph.number_of_nodes()))
print("Number of Edges:", str(grid.graph.number_of_edges()))
print("Number of Riders:", len(riders))
print("Number of Drivers:", len(drivers))
print("Number of Ride Requests:", len(ride_requests))
print("\n")

event_queue = initialize_event_queue(ride_requests)
simulation = Simulation(riders, drivers, ride_requests, grid)

process_events(event_queue, simulation)

Grid dimensions: 15 15
Number of Zones: 225
Number of Nodes: 225
Number of Edges: 840
Number of Riders: 2064
Number of Drivers: 516
Number of Ride Requests: 2064


State recorded
State recorded
State recorded
State recorded
State recorded
State recorded
Processing event: Event(t=6, action='Request Ride', spaces={'ride_id': 1002}) at time 6
Handling ride request event for ride 1002 at time 6
Travel time: 80, Cost: 80.0
Driver Driver 0 assigned to ride 1002 at time 86
State recorded
Processing event: Event(t=7, action='Request Ride', spaces={'ride_id': 281}) at time 7
Handling ride request event for ride 281 at time 7
Travel time: 40, Cost: 40.0
Driver Driver 1 assigned to ride 281 at time 47
State recorded
Processing event: Event(t=8, action='Request Ride', spaces={'ride_id': 289}) at time 8
Handling ride request event for ride 289 at time 8
Travel time: 35, Cost: 35.0
Driver Driver 2 assigned to ride 289 at time 43
State recorded
Processing event: Event(t=9, action='Request Ride', spac

In [21]:
# Verify that all the initial ride requests were completed
if len(ride_requests) == len(simulation.completed_rides):
    print("Verification successful: All requested rides were completed.")
else:
    print(f"Verification failed: {len(ride_requests)} requests, {len(simulation.completed_rides)} completed.")

Verification successful: All requested rides were completed.


### Data Written to CSV in the StateMap

The `StateMap` class in the script is responsible for recording and writing the state of the simulation at each time step to a CSV file. The data recorded includes key metrics that help evaluate the performance of the ridesharing system. Here is an explanation of the data written to the CSV file:

1. **Time (`time`)**:
   - This column represents the current time step in the simulation. It helps in understanding the progression of events over time and allows for temporal analysis of the system's performance.

2. **Completed Rides (`completed_rides`)**:
   - This column records the cumulative number of rides that have been completed up to the current time step. It is a critical metric for evaluating the effectiveness and efficiency of the ridesharing system in fulfilling ride requests.

3. **Total Distance (`total_distance`)**:
   - This column sums the distances traveled by all completed rides up to the current time step. It provides insights into the total distance covered by drivers in the system and can be used to assess fuel consumption, wear and tear on vehicles, and overall system efficiency.

4. **Total Cost (`total_cost`)**:
   - This column records the cumulative cost incurred by all completed rides up to the current time step. The cost is typically calculated based on the distance traveled and can include factors like fare rates, congestion pricing, and dynamic pricing adjustments.


In [25]:
file_path = 'state_map_dipr.csv'

simulation.state_map.write_to_csv(file_path)

# Read the CSV file and display the first 100 rows
df = pd.read_csv(file_path)
time_units = 100 # represents steps in the system
df.head(time_units)


,time,completed_rides,total_distance,total_cost
0,0,0,0,0.0
1,1,0,0,0.0
2,2,0,0,0.0
3,3,0,0,0.0
4,4,0,0,0.0
...,...,...,...,...
95,95,10,180,450.0
96,96,12,252,630.0
97,97,12,252,630.0
98,98,12,252,630.0


In [26]:
df.describe()


,time,completed_rides,total_distance,total_cost
count,2158.000000,2158.000000,2158.000000,2158.000000
mean,1078.500000,1009.853568,36708.916589,91772.291474
std,623.105261,628.217336,23007.164208,57517.910519
min,0.000000,0.000000,0.000000,0.000000
25%,539.250000,462.250000,16628.750000,41571.875000
50%,1078.500000,1012.000000,36907.000000,92267.500000
75%,1617.750000,1548.500000,56330.750000,140826.875000
max,2157.000000,2064.000000,75590.000000,188975.000000
